<h1>Applied Data Science Capstone<h1>

This notebook will be used mainly for the applied data science capstone project thats part of the <b>IBM Data Science Professional Certificate</b> course.





In [2]:
import numpy as np
import pandas as pd
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
